<a href="https://colab.research.google.com/github/nallagondu/DATASCIENCE-practice/blob/main/HTTP_Log_Analysis_Via_Spark_gangi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/content'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/content/TestFileGenerator.py
/content/logfiles.log
/content/.config/.last_survey_prompt.yaml
/content/.config/active_config
/content/.config/config_sentinel
/content/.config/gce
/content/.config/default_configs.db
/content/.config/.last_opt_in_prompt.yaml
/content/.config/.last_update_check.json
/content/.config/configurations/config_default
/content/.config/logs/2023.12.18/14.23.38.210663.log
/content/.config/logs/2023.12.18/14.23.24.153475.log
/content/.config/logs/2023.12.18/14.22.34.853310.log
/content/.config/logs/2023.12.18/14.23.01.527695.log
/content/.config/logs/2023.12.18/14.23.15.463718.log
/content/.config/logs/2023.12.18/14.23.39.295290.log
/content/sample_data/anscombe.json
/content/sample_data/README.md
/content/sample_data/mnist_train_small.csv
/content/sample_data/mnist_test.csv
/content/sample_data/california_housing_test.csv
/content/sample_data/california_housing_train.csv


**Install PySpark¶**


In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=6b54e4e8c9ebf28d711cc2c5f0579dfd6055dc3f7dd251cab10d9d573ae5195b
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


**Initialize the pyspark environment, load the text log file, parse the file so we have structure**

In [ ]:
import pyspark
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import Row


appName = 'process-log-files'
path = "/content/logfiles.log"


spark = SparkSession \
    .builder \
    .appName(appName) \
    .getOrCreate()

def parseLine(row):
    line = row.value
    tokens = line.split()
    ip = tokens[0]
    method = tokens[5][1:]
    path = tokens[6]
    status = tokens[8]
    endPoint = f"{method} {path}"
    return Row(ip=ip, method=method, path=path, status=status, endPoint=endPoint)


parsedRDD = spark.read.text(path) \
    .rdd \
    .map(parseLine)

In [ ]:
parsedRDD.count()

25993

**Group all requests by the IP address, and find the IPs that send out most requests.**

In [ ]:
ipCountRddSorted = parsedRDD.groupBy(lambda r: r.ip)\
    .map(lambda r: (r[0], len(list(r[1])))) \
    .sortBy(lambda r: r[1], ascending=False)

ipCountRddSorted.take(5)

[('162.253.4.179', 1),
 ('160.36.208.51', 1),
 ('59.107.116.6', 1),
 ('137.196.118.126', 1),
 ('144.140.97.239', 1)]

As the result shows, each IP sends out at most 2 requests, we do not have a lot of requests from certain IPs.
Now let's find the most popular endpoints, and their traffic bold text

In [ ]:
endpointCountRddSorted = parsedRDD.groupBy(lambda r: r.endPoint)\
    .map(lambda r: (r[0], len(list(r[1])))) \
    .sortBy(lambda r: r[1], ascending=False)

In [ ]:
top5Endpoints = endpointCountRddSorted.take(5)
print(top5Endpoints)

[('POST /usr', 1372), ('POST /usr/admin/developer', 1355), ('PUT /usr/admin/developer', 1343), ('DELETE /usr/admin', 1340), ('PUT /usr/register', 1326)]


In [ ]:
top5IP = ipCountRddSorted.take(5)
top5IP

[('162.253.4.179', 1),
 ('160.36.208.51', 1),
 ('59.107.116.6', 1),
 ('137.196.118.126', 1),
 ('144.140.97.239', 1)]

In [ ]:
spark = SparkSession \
    .builder \
    .appName(appName) \
    .getOrCreate()

In [ ]:
spark.sparkContext.getConf().getAll()

[('spark.app.name', 'process-log-files'),
 ('spark.driver.host', 'e98ea3aa8c5e'),
 ('spark.executor.id', 'driver'),
 ('spark.sql.warehouse.dir', 'file:/content/spark-warehouse'),
 ('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDir

In [ ]:
path = "/content/logfiles.log"
df1 = spark.read.text(path)
df1.show()

+--------------------+
|               value|
+--------------------+
|233.223.117.90 - ...|
|162.253.4.179 - -...|
|252.156.232.172 -...|
|182.215.249.159 -...|
|160.36.208.51 - -...|
|255.231.52.33 - -...|
|238.217.83.154 - ...|
|119.170.1.203 - -...|
|59.107.116.6 - - ...|
|137.196.118.126 -...|
|26.167.128.186 - ...|
|144.140.97.239 - ...|
|86.194.222.239 - ...|
|254.54.186.144 - ...|
|55.25.7.93 - - [2...|
|154.131.45.155 - ...|
|207.194.20.187 - ...|
|202.8.213.171 - -...|
|26.44.136.193 - -...|
|217.20.3.105 - - ...|
+--------------------+
only showing top 20 rows



In [ ]:
type(df1)
type(df1.rdd)

pyspark.rdd.RDD

In [ ]:
rdd = df1.rdd

In [ ]:
rdd.first()

Row(value='233.223.117.90 - - [27/Dec/2037:12:00:00 +0530] "DELETE /usr/admin HTTP/1.0" 502 4963 "-" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4380.0 Safari/537.36 Edg/89.0.759.0" 45')

**Parse each line of log into structure data¶**

In [ ]:
from pyspark.sql import Row
class LogLine:
    def __init__(self, line):
        #print(type(line))
        self.parse(line)
        pass

    def parse(self, line):
            tokens = line.split()
            self.ip = tokens[0]
            self.method = tokens[5][1:]
            self.path = tokens[6]
            self.status = tokens[8]
            self.endPoint = f"{self.method} {self.path}"


#             print(tokens)
#             print(self.method)
#             print(self.path)
#             print(self.status)

    def asRow(self):
        return Row(ip=self.ip, method=self.method, path=self.path, status=self.status, endPoint=self.endPoint)

def parseLine(line):
    return LogLine(line).asRow()


# line = '233.223.117.90 - - [27/Dec/2037:12:00:00 +0530] "DELETE /usr/admin HTTP/1.0" 502 4963 "-" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4380.0 Safari/537.36 Edg/89.0.759.0" 45'

# logLine = LogLine(line)


In [ ]:
parsedRdd = rdd.map(lambda x: parseLine(x.value))

parsedRdd

PythonRDD[53] at RDD at PythonRDD.scala:53

**Group by IP, and find the top IP**

In [21]:
ipCountRddSorted = parsedRdd.groupBy(lambda r: r.ip)\
    .map(lambda r: (r[0], len(list(r[1])))) \
    .sortBy(lambda r: r[1], ascending=False)

In [22]:
top40IPs = ipCountRddSorted.take(40)
print(top40IPs)

[('162.253.4.179', 1), ('160.36.208.51', 1), ('59.107.116.6', 1), ('137.196.118.126', 1), ('144.140.97.239', 1), ('86.194.222.239', 1), ('254.54.186.144', 1), ('207.194.20.187', 1), ('149.194.199.18', 1), ('115.101.165.251', 1), ('211.69.213.210', 1), ('125.87.60.188', 1), ('183.129.168.199', 1), ('102.247.49.87', 1), ('230.230.213.17', 1), ('148.5.169.251', 1), ('90.64.62.239', 1), ('97.15.37.214', 1), ('159.238.93.133', 1), ('223.254.74.157', 1), ('66.181.188.94', 1), ('228.181.201.16', 1), ('172.175.150.80', 1), ('40.129.224.59', 1), ('183.51.44.206', 1), ('190.247.167.164', 1), ('183.155.98.170', 1), ('142.20.69.223', 1), ('157.44.23.213', 1), ('136.170.251.93', 1), ('85.2.215.227', 1), ('214.60.60.96', 1), ('93.234.0.101', 1), ('132.176.187.103', 1), ('213.127.117.32', 1), ('220.194.253.161', 1), ('247.61.75.27', 1), ('103.176.203.18', 1), ('68.226.160.154', 1), ('112.164.86.231', 1)]


In [23]:
endpointCountRddSorted = parsedRdd.groupBy(lambda r: r.endPoint)\
    .map(lambda r: (r[0], len(list(r[1])))) \
    .sortBy(lambda r: r[1], ascending=False)

In [24]:
top40Endpoints = endpointCountRddSorted.take(40)
print(top40Endpoints)

[('POST /usr', 1372), ('POST /usr/admin/developer', 1355), ('PUT /usr/admin/developer', 1343), ('DELETE /usr/admin', 1340), ('PUT /usr/register', 1326), ('GET /usr/register', 1326), ('GET /usr/admin/developer', 1324), ('DELETE /usr/admin/developer', 1309), ('PUT /usr/login', 1299), ('PUT /usr/admin', 1296), ('GET /usr/login', 1296), ('DELETE /usr', 1294), ('GET /usr', 1290), ('PUT /usr', 1288), ('DELETE /usr/login', 1284), ('GET /usr/admin', 1270), ('POST /usr/register', 1265), ('DELETE /usr/register', 1265), ('POST /usr/login', 1256), ('POST /usr/admin', 1195)]
